# Generador de pokemons

Este script simulará la aparición de pokemons de una manera pseudoaleatoria. Los pokemos se pueden repetir, aparecerán aproximadamente cada `MEAN_INTERVAL` segundos, y aparecerán en el entorno de las coordenadas definidas en `COORDINATES`.

In [ ]:
import pandas as pd
from confluent_kafka import Producer
import time
import json
import random
from ejercicios.pokemons import SEED, TOPIC_POKEMONS, DATA, COORDINATES, MEAN_INTERVAL

In [ ]:
random.seed(SEED)

In [ ]:
ds = pd.read_csv(DATA)
ds = ds.drop(labels=['#', 'Type 2', 'Total', 'Sp. Atk', 'Sp. Def', 'Generation'], axis=1)
ds = ds.rename({'Type 1': 'Type'}, axis=1)
ds.head()

In [ ]:
pokemon = ds.sample(n=1)
as_dict = pokemon.to_dict('records')[0]

In [ ]:
pokemon

In [ ]:
as_dict

In [ ]:
GAUSS_LAT_MADRID = COORDINATES['GAUSS_LAT_MADRID']
GAUSS_LON_MADRID = COORDINATES['GAUSS_LON_MADRID']
GAUSS_LAT_SEGOVIA = COORDINATES['GAUSS_LAT_SEGOVIA']
GAUSS_LON_SEGOVIA = COORDINATES['GAUSS_LON_SEGOVIA']


In [ ]:
as_dict['lat'] = random.gauss(**GAUSS_LAT_MADRID)
as_dict['lon'] = random.gauss(**GAUSS_LON_MADRID)
as_dict

In [ ]:
json.dumps(as_dict)

In [ ]:
TOPIC_POKEMONS

In [ ]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        pass
        # print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

while True:
    pokemon = ds.sample(n=1)
    pokemon = pokemon.to_dict('records')[0]

    if random.randrange(0, 10, 1) < 5:
        pokemon['lat'] = random.gauss(**GAUSS_LAT_MADRID)
        pokemon['lon'] = random.gauss(**GAUSS_LON_MADRID)
    else:
        pokemon['lat'] = random.gauss(**GAUSS_LAT_SEGOVIA)
        pokemon['lon'] = random.gauss(**GAUSS_LON_SEGOVIA)
        

    # execute any available delivery report callbacks from previous produce() calls
    p.poll(0)

    # Asynchronously produce a message, the delivery report callback
    # will be triggered from poll() above, or flush() below, when the message has
    # been successfully delivered or failed permanently.
    print("Sending pokemon {Name}:{Type}:{lat}:{lon}".format(**pokemon))
    p.produce(
        TOPIC_POKEMONS,
        json.dumps(pokemon).encode('utf-8'),
        callback=delivery_report
    )
    time.sleep(abs(random.gauss(MEAN_INTERVAL, 1)))

# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()
